In [1]:
import numpy as np

In [ ]:
from turtle import backward


class VectorValue:
    def __init__(self, data, _children = (), _op: str = "", label: str = ""):
        self.data = list(data)
        self.grad = [0.0 for _ in self.data]

        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"VectorValue(data = {self.data})"
    
    # Addition(s)
    def __add__(self, other):
        other = other if isinstance(other, VectorValue) else VectorValue(other)
        assert len(self.data) == len(other.data)

        out_data = [a + b for a, b in zip(self.data, other.data)]
        out = VectorValue(out_data, (self, other), "+")

        def _backward():
            for i in range(len(self.grad)):
                self.grad[i] += out.grad
                other.grad[i] += out.grad
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other
    
    # Multiplication
    def __mul__(self, other):
        other = other if isinstance(other, VectorValue) else VectorValue(other)
        assert len(other.data) = len(self.data)
        out_data = [a * b for a, b in zip(self.data, other.data)]
        out = VectorValue(out_data, (self, other), "*")

        def _backward():
            for i in range(len(self.grad)):
                self.grad[i] += other.data[i] * out.grad[i]
                other.grad[i] += self.data[i] * out.grad[i]
        
        out._backward = _backward
        return out
    
    def __rmul__(self, other):
        return self * other
             
    
    # Negation
    def __neg__(self):
        return self * -1
    
    # Subtractions
    
    def __sub__(self, other):
        return self + (-other)
    
    def __rsub__(self, other):
        return other + (-self)

        


    

In [9]:
a = VectorValue([1, 2, 3])
b = [1, 2, 3]
a+b

VectorValue(data = [1, 2, 3, 1, 2, 3])